<a href="https://colab.research.google.com/github/CalvHobbes/ai-agents/blob/main/Agentic_RAG_with_LLamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## REF - https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/2/router-query-engine

In [1]:
# Download the file dynamically from GitHub
!wget -O agentic_rag_llamaindex_requirements.txt https://raw.githubusercontent.com/CalvHobbes/ai-agents/main/agentic_rag_llamaindex_requirements.txt




--2025-01-22 11:14:27--  https://raw.githubusercontent.com/CalvHobbes/ai-agents/main/agentic_rag_llamaindex_requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252 [text/plain]
Saving to: ‘agentic_rag_llamaindex_requirements.txt’

agentic_rag_llamain 100%[===================>]     252  --.-KB/s    in 0s      

2025-01-22 11:14:28 (3.74 MB/s) - ‘agentic_rag_llamaindex_requirements.txt’ saved [252/252]



In [2]:
# Install the requirements
!pip install -r agentic_rag_llamaindex_requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00


In [3]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2025-01-22 11:27:47--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  45.9MB/s    in 0.4s    

2025-01-22 11:27:47 (45.9 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [7]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)


In [5]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI_KEY')

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [10]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [13]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [14]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [16]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [17]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through role specialization, workflow management, and efficient communication mechanisms. It incorporates an executable feedback mechanism to improve code generation quality during runtime and achieves state-of-the-art performance on various benchmarks. The document also provides insights into the development process of a software application called the "Drawing App" using MetaGPT, highlighting stages like requirement gathering, UI design, system architecture, implementation approach, testing, and task allocation. It discusses the use of Python libraries, creation of a user-friendly GUI, code generation, unit testing, and performance evaluation. Additionally, it touches upon challenges, ethical concerns, and the impact of MetaGPT o

In [20]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the MetaGPT paper, which may contain information on how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. By storing information in this global message pool, agents can retrieve required information without the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


In [21]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results show that MetaGPT effectively addresses challenges related to context utilization, code hallucinations, and information overload in the development of recommendation engines. By accurately unfolding natural language descriptions, maintaining information validity, and focusing on granular tasks like requirement analysis, MetaGPT mitigates issues such as ambiguity, incomplete code implementation, and irrelevant information. The use of a global message pool and subscription mechanism helps streamline communication and filter out irrelevant contexts, enhancing the efficiency and utility of the information provided.


In [22]:
response = query_engine.query("summarise ablation study")
print(str(response))

Selecting query engine 0: Ablation study is often used for summarization questions related to the performance of different components in a model like MetaGPT..
The ablation study conducted in the MetaGPT project focused on systematically excluding different roles within the multi-agent system to evaluate their impact on the final results. The study demonstrated that the addition of specialized roles consistently improved the number of revisions needed and the executability of the generated code. Introducing more roles led to improved performance metrics, despite a slight increase in expenses, emphasizing the effectiveness of having diverse roles with specific expertise in the software development process within the MetaGPT framework.
